In [2]:
from goal_rec_utils.plan import Plan
from goal_rec_utils.plan_generator import PlanGeneratorMultiPerc
from utils_unibs.files import load_from_folder
import numpy as np
import goal_rec_utils
from tensorflow.keras.utils import Sequence
from incremental_model_training import SaveBestModelCallback, run_tests, Custom_Hamming_Loss1
from tensorflow.keras.models import load_model
import incremental_model_training
import numpy as np
from tensorflow.keras import metrics
from tensorflow.keras.models import Model
import datetime
from os import path
import os





In [ ]:
class PlanGeneratorMultiPerc(Sequence):
    def __getitem__(self, index):
        batches = self.plans[index * self.batch_size:(index + 1) * self.batch_size]
        X = np.zeros((int(self.batch_size), int(self.max_dim)))
        Y = np.zeros((int(self.batch_size), len(self.dizionario_goal)))
        for i, plan in enumerate(batches):
            seed = plan.plan_name.rsplit('-p',1)[1]
            seed = seed.split('_', 1)[0]
            seed = seed.rsplit('.', 1)[0]
            np.random.seed(int(seed))
            p = np.random.uniform(self.min_perc, self.perc)
            actions = get_actions(plan.actions, p, self.dizionario)
            fill_action_sequence(X, self.max_dim, actions, i)
            Y[i] = get_goal(plan.goals, self.dizionario_goal)
        return X, Y

    def __len__(self):
        return len(self.plans) // self.batch_size

    def __init__(self, plans, dizionario, dizionario_goal, dizionario_new_goal, batch_size, max_dim, min_perc, max_perc, shuffle=True):
        self.plans = plans
        self.dizionario_goal = dizionario_goal
        self.fizionario_new_goal = dizionario_new_goal
        self.dizionario = dizionario
        self.batch_size = batch_size
        self.max_dim = max_dim
        self.min_perc = min_perc
        self.perc = max_perc
        self.shuffle = shuffle

    def on_epoch_end(self):
        '''Updates indexes after each epoch'''
        if self.shuffle == True:
            np.random.shuffle(self.plans)

def get_actions(actions: list, perc: float, dizionario: dict):
    '''
    Get a sub-sequence made by a given percentage of elements in action.
    Args:
        actions: a list that contains the actions as strings
        perc: a float that represents the percentage of actions to keep. It must be from 0 to 1 included.
        dizionario: a dictionary that contains all the action labels and their corresponding unique indexes
    Returns:
        A list that contains the indexes of the actions in the sub-sequence
    '''
    if actions is None or len(actions) == 0:
        return []
    if perc > 1:
        perc = 1
    elif perc < 0:
        perc = 0
    size = int(np.ceil(len(actions) * perc))
    if size == 0:
        size = 1
    indexes = np.ones(size, dtype=int) * -1
    i = 0
    ind_list = list(range(len(actions)))
    np.random.shuffle(ind_list)
    while i < size:
        ind = ind_list.pop(0)
        if ind not in indexes:
            indexes[i] = ind
            i += 1
    indexes = np.sort(indexes)
    return [dizionario[a.name] for a in np.take(actions, indexes)]


def fill_action_sequence(X, max_dim, actions, i):
    for j in range(max_dim):
        if j < len(actions):
            X[i][j] = actions[j]
        else:
            if type(actions[0]) == int:
                X[i][j] = 0
            else:
                X[i][j] = np.zeros(shape=(len(actions[0]),))

def get_goal(g, dizionario_goal):
    goal = np.zeros(len(dizionario_goal))
    for subgoal in g:
        if subgoal in dizionario_goal:
            goal = goal + dizionario_goal[subgoal]
    return goal

def get_seed(string: str):
    '''
    Turns a string into an int that can be used as a seed
    Args:
        string: the string to transform in seed
    Returns:
        an integer containing the seed
    '''
    seed = 0
    for c in string:
        seed += ord(c)
    return seed


incremental_model_training.PlanGeneratorMultiPerc = PlanGeneratorMultiPerc

In [3]:
[plans] = load_from_folder('/data/users/mchiari/WMCA/datasets/satellite/optimal_plans/plans_max-plan-dim=30_train_percentage=0.8', ['train_plans'])
[goals_dict, actions_dict] = load_from_folder('/data/users/mchiari/WMCA/datasets/satellite/optimal_plans/dictionaries_and_plans', ['dizionario_goal', 'dizionario'])

train_plans loaded from /data/users/mchiari/WMCA/datasets/satellite/optimal_plans/plans_max-plan-dim=30_train_percentage=0.8
dizionario_goal loaded from /data/users/mchiari/WMCA/datasets/satellite/optimal_plans/dictionaries_and_plans
dizionario loaded from /data/users/mchiari/WMCA/datasets/satellite/optimal_plans/dictionaries_and_plans


In [ ]:
def create_model(generator: PlanGeneratorMultiPerc, lr: float):
    
    

    input_layer = Input(shape=(generator.max_dim,))
    embedding_layer = Embedding(input_dim=len(generator.dizionario)+1,
                                input_length=generator.max_dim, 
                                output_dim=82,
                                mask_zero=True,
                                name='embedding')(input_layer)
    lstm_layer = LSTM(363, return_sequences=True, dropout=0, recurrent_dropout=0.21543712857716188, activation='linear', name='lstm')(embedding_layer)
    attention_weights = AttentionWeights(generator.max_dim, name='attention_weights')(lstm_layer)
    context_vector = ContextVector()([lstm_layer, attention_weights])
    output_layer = Dense(len(generator.dizionario_goal), activation='sigmoid', name='dense')(context_vector)
    model = Model(inputs=input_layer, outputs=output_layer)
    optimizer = Adam(learning_rate=lr)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', Custom_Hamming_Loss1, metrics.Precision(name='precision')])

In [4]:
def create_goals_dict(goals: list, goals_dict: dict = {}):
    new_goals_dict = goals_dict.copy()
    new_goals = set(goals).difference(set(goals_dict.keys()))
    for k in new_goals_dict:
        new_goals_dict[k] = np.append(new_goals_dict[k], np.zeros([len(new_goals),])) 
    for g in new_goals:
        l = np.zeros([len(goals_dict)+len(new_goals),])
        l[len(new_goals_dict)] = 1
        new_goals_dict[g] = l
    print(f'{len(new_goals_dict) = }')
    return new_goals_dict

def create_actions_dict(subplans: list, actions_dict: dict = {}):
    for p in subplans:
        for a in p.actions:
            if a.name not in actions_dict:
                actions_dict[a.name] = len(actions_dict)+1
    print(f'{len(actions_dict) = }')
    return actions_dict

In [5]:
def create_subplans(all_plans: list, sub_goals_dict: dict, old_sub_goals_dict: dict):
    subplans = set()
    count = 0
    print(f'{sub_goals_dict.keys() = }, {old_sub_goals_dict.keys() = }')
    new_sub_goals = set(sub_goals_dict.keys()).difference(set(old_sub_goals_dict.keys()))
    for goal_fact in new_sub_goals:
        for p in all_plans:
            if goal_fact in p.goals:
                subplans.add(p)
                count += 1
    print(f'{len(subplans) = }, {count = }')  
    return subplans

def extend_subplans(subplans: set, old_subplans: set, old_goals: list):
    to_add = set()
    for g in old_goals:
        count = 0
        for p in list(old_subplans):
            if count == 100:
                break
            if g in p.goals and p not in subplans and p not in to_add:
                to_add.add(p)
                count += 1
            elif g in p.goals and p not in subplans:
                count += 1
        print(f'{g} : {len(to_add) = }, {count = }')
    return to_add      
    

In [6]:
def create_test_val_plans(subplans: list, perc: float = 0.85):
    np.random.seed(420)
    np.random.shuffle(subplans)
    train_plans = subplans[:int(len(subplans)*perc)]
    val_plans = subplans[int(len(subplans)*perc):]
    print(f'{len(train_plans) = }, {len(val_plans) = }')
    return train_plans, val_plans

In [7]:
iteration = 0
target_dir = path.join('/data/users/mchiari/WMCA/satellite/transfer_learning/', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
temp_dir = path.join(target_dir, f'temp_dir')
os.makedirs(temp_dir, exist_ok=True)
models_dir = path.join(target_dir, f'models')
os.makedirs(models_dir, exist_ok=True)

In [8]:
def preprocess(sub_goals: list, old_sub_plans: set, old_sub_goals_dict: dict, old_sub_actions_dict: dict):
    sub_goals_dict = create_goals_dict(sub_goals, old_sub_goals_dict)
    sub_plans = create_subplans(plans, sub_goals_dict, old_sub_goals_dict)
    to_add = extend_subplans(sub_plans, old_sub_plans, list(old_sub_goals_dict.keys()))
    sub_plans = sub_plans.union(to_add)
    sub_actions_dict = create_actions_dict(sub_plans, old_sub_actions_dict)
    train_plans, val_plans = create_test_val_plans(list(sub_plans), 0.85)
    train_generator = PlanGeneratorMultiPerc(train_plans, sub_actions_dict, sub_goals_dict, 64, 30, 0.3, 1)
    val_generator = PlanGeneratorMultiPerc(val_plans, sub_actions_dict, sub_goals_dict, 64, 30, 0.3, 1)
    return train_generator, val_generator, sub_goals_dict, sub_actions_dict, train_plans, val_plans

def test_model(model, sub_goals_dict, sub_actions_dict, val_plans, iteration: int):
    run_tests(model, val_plans, sub_actions_dict, sub_goals_dict, 64, 30, 0.3, 1, None)
    model.save(path.join(models_dir, f'model_{iteration}.h5'))

def run(iteration: int, sub_goals: list, old_sub_plans: set, old_sub_goals_dict: dict, old_sub_actions_dict: dict):
    print(f'{old_sub_goals_dict.keys() = }')
    train_generator, val_generator, sub_goals_dict, sub_actions_dict, train_plans, val_plans = preprocess(sub_goals, old_sub_plans, old_sub_goals_dict, old_sub_actions_dict)
    model = create_model(train_generator, 0.001)
    model.fit(train_generator, epochs=100, validation_data=val_generator, callbacks=[SaveBestModelCallback(temp_dir, 5, 0)])
    model = load_model(path.join(temp_dir, 'model.h5'), custom_objects = {'AttentionWeights' : goal_rec_utils.attention_layers.AttentionWeights, 'ContextVector' : goal_rec_utils.attention_layers.ContextVector, 'Custom_Hamming_Loss1' : Custom_Hamming_Loss1})
    test_model(model, sub_goals_dict, sub_actions_dict, val_plans, iteration)
    return train_plans, val_plans, sub_goals_dict, sub_actions_dict

def transfer_model(model: Model, old_model: Model):
    old_w = old_model.get_layer('embedding').get_weights()[0]
    w = model.get_layer('embedding').get_weights()[0]
    for i in range(len(old_w)):
        w[i] = old_w[i]
    model.get_layer('embedding').set_weights([w])
    model.get_layer('lstm').set_weights(old_model.get_layer('lstm').get_weights())
    model.get_layer('attention_weights').set_weights(old_model.get_layer('attention_weights').get_weights())
    w = model.get_layer('dense').get_weights()
    old_w = old_model.get_layer('dense').get_weights()
    for i in range(len(w[0])):
        w[0][i, :-5] = old_w[0][i]
    w[1][:-5] = old_w[1]
    model.get_layer('dense').set_weights(w)
    return model

In [9]:
sub_goals = list(goals_dict.keys())[0:5]
train_plans, val_plans, sub_goals_dict, sub_actions_dict = run(iteration, sub_goals, {}, {}, {})

old_sub_goals_dict.keys() = dict_keys([])
len(new_goals_dict) = 5
sub_goals_dict.keys() = dict_keys(['have_image planet3 infrared0', 'have_image planet5 infrared3', 'have_image phenomenon14 infrared0', 'have_image phenomenon4 infrared0', 'have_image star5 infrared3']), old_sub_goals_dict.keys() = dict_keys([])
len(subplans) = 2576, count = 2660
len(actions_dict) = 15973
len(train_plans) = 2189, len(val_plans) = 387


2023-05-09 13:33:07.396940: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 13:33:07.875510: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-05-09 13:33:07.876606: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2294605000 Hz


Epoch 1/100
34/34 [==============================] - 9s 177ms/step - loss: 0.6375 - accuracy: 0.2134 - Custom_Hamming_Loss1: 0.2443 - precision: 0.2164 - val_loss: 0.5321 - val_accuracy: 0.2005 - val_Custom_Hamming_Loss1: 0.2068 - val_precision: 0.0000e+00
Epoch 2/100
34/34 [==============================] - 4s 131ms/step - loss: 0.5215 - accuracy: 0.2348 - Custom_Hamming_Loss1: 0.2073 - precision: 0.0000e+00 - val_loss: 0.5136 - val_accuracy: 0.2031 - val_Custom_Hamming_Loss1: 0.2068 - val_precision: 0.0000e+00
Epoch 3/100
34/34 [==============================] - 4s 130ms/step - loss: 0.5021 - accuracy: 0.3485 - Custom_Hamming_Loss1: 0.2064 - precision: 0.3657 - val_loss: 0.4991 - val_accuracy: 0.3724 - val_Custom_Hamming_Loss1: 0.2068 - val_precision: 0.0000e+00
New best model found with loss 0.4991004467010498
Epoch 4/100
34/34 [==============================] - 4s 131ms/step - loss: 0.3970 - accuracy: 0.8050 - Custom_Hamming_Loss1: 0.1775 - precision: 0.7288 - val_loss: 0.5127 - va

/opt/anaconda/anaconda3/envs/goal_rec/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
old_sub_goals_dict = sub_goals_dict
old_sub_actions_dict = sub_actions_dict
old_train_plans = train_plans
old_val_plans = val_plans

In [14]:
sub_goals = list(goals_dict.keys())[5:10]
train_generator, val_generator, sub_goals_dict, sub_actions_dict, train_plans, val_plans = preprocess(sub_goals, set(old_train_plans).union(set(old_val_plans)), old_sub_goals_dict, old_sub_actions_dict)
model = create_model(train_generator, 1e-4)
old_model = load_model(path.join(models_dir, 'model_0.h5'), custom_objects = {'AttentionWeights' : goal_rec_utils.attention_layers.AttentionWeights, 'ContextVector' : goal_rec_utils.attention_layers.ContextVector, 'Custom_Hamming_Loss1' : Custom_Hamming_Loss1})
transfer_model(model, old_model)
model.fit(train_generator, epochs=100, validation_data=val_generator, callbacks=[SaveBestModelCallback(temp_dir, 25, 0)])
test_model(model, sub_goals_dict, sub_actions_dict, train_plans, 1)
test_model(model, sub_goals_dict, sub_actions_dict, val_plans, 1)


len(new_goals_dict) = 10
sub_goals_dict.keys() = dict_keys(['have_image planet3 infrared0', 'have_image planet5 infrared3', 'have_image phenomenon14 infrared0', 'have_image phenomenon4 infrared0', 'have_image star5 infrared3', 'have_image planet3 image3', 'have_image phenomenon4 image3', 'have_image groundstation2 image0', 'pointing satellite4 planet5', 'pointing satellite3 planet9']), old_sub_goals_dict.keys() = dict_keys(['have_image planet3 infrared0', 'have_image planet5 infrared3', 'have_image phenomenon14 infrared0', 'have_image phenomenon4 infrared0', 'have_image star5 infrared3'])
len(subplans) = 2439, count = 2495
have_image planet3 infrared0 : len(to_add) = 100, count = 100
have_image planet5 infrared3 : len(to_add) = 200, count = 100
have_image phenomenon14 infrared0 : len(to_add) = 294, count = 100
have_image phenomenon4 infrared0 : len(to_add) = 385, count = 100
have_image star5 infrared3 : len(to_add) = 478, count = 100
len(actions_dict) = 22478
len(train_plans) = 2479, l

/opt/anaconda/anaconda3/envs/goal_rec/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.13363486842105263

Hamming Loss: 0.09013157894736842

                                   precision    recall  f1-score   support

     have_image planet3 infrared0       1.00      0.72      0.83        95
     have_image planet5 infrared3       0.98      0.85      0.91       113
have_image phenomenon14 infrared0       0.99      0.79      0.88       102
 have_image phenomenon4 infrared0       0.97      0.75      0.85        99
       have_image star5 infrared3       1.00      0.78      0.88       101
        have_image planet3 image3       0.00      0.00      0.00       431
    have_image phenomenon4 image3       0.68      0.03      0.05       488
 have_image groundstation2 image0       0.00      0.00      0.00       448
      pointing satellite4 planet5       0.33      0.01      0.02       364
      pointing satellite3 planet9       0.00      0.00      0.00       345

                        micro avg       0.95      0.16      0.27      2586
                        macro av

/opt/anaconda/anaconda3/envs/goal_rec/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda/anaconda3/envs/goal_rec/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
print(sub_goals_dict)
print(old_sub_goals_dict)

{'have_image planet5 infrared3': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'have_image planet3 infrared0': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]), 'have_image phenomenon4 infrared0': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]), 'have_image star5 infrared3': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]), 'have_image phenomenon14 infrared0': array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]), 'have_image planet3 image3': array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), 'have_image phenomenon4 image3': array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]), 'have_image groundstation2 image0': array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]), 'pointing satellite4 planet5': array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]), 'pointing satellite3 planet9': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])}
{'have_image planet5 infrared3': array([1., 0., 0., 0., 0.]), 'have_image planet3 infrared0': array([0., 1., 0., 0., 0.]), 'have_image phenomenon4 infrared0': array([0., 0., 1., 0., 

In [25]:
print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 82)       1843278     input_4[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 30, 363)      647592      embedding[0][0]                  
__________________________________________________________________________________________________
attention_weights (AttentionWei (None, 30)           393         lstm[0][0]                       
____________________________________________________________________________________________

In [27]:
print(model.get_layer('dense').get_weights()[1])
print(old_model.get_layer('dense').get_weights()[1])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[-0.06072112 -0.05820167 -0.05302558 -0.03600901 -0.07074687]


In [52]:
s = set()
goals = list(goals_dict.keys())[5:10]
for i in range(5,10):
    k = list(goals_dict.keys())[i]
    for p in plans:
        if k in p.goals:
            s.add(p)
            count += 1
print(len(s), count)


2439 5155


In [53]:
old_goals = list(goals_dict.keys())[0:5]
to_add = set()
for g in old_goals:
    count = 0
    for p in list(old_set):
        if count == 100:
            break
        if g in p.goals and p not in s and p not in to_add:
            to_add.add(p)
            count += 1
        elif g in p.goals and p not in s:
            count += 1
    print(len(to_add), count)

print(len(s), len(to_add))
s = s.union(to_add)
print(len(s))
        

100 100
200 100
295 100
391 100
489 100
2439 489
2928


In [72]:
sub_plans = list(s)
np.random.shuffle(sub_plans)
train_plans = list(s)[:int(len(s)*0.85)]
val_plans = list(s)[int(len(s)*0.85):]
sub_goals_dict = create_goals_dict(list(goals_dict.keys())[0:10])
print(sub_goals_dict)
sub_actions_dict = create_actions_dict(list(s))
print(len(sub_actions_dict))
train_generator = PlanGeneratorMultiPerc(train_plans, sub_actions_dict, sub_goals_dict, 64, 30, 0.3, 1)
val_generator = PlanGeneratorMultiPerc(val_plans, sub_actions_dict, sub_goals_dict, 64, 30, 0.3, 1)

{'have_image phenomenon14 infrared0': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'have_image star5 infrared3': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]), 'have_image planet3 infrared0': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]), 'have_image phenomenon4 infrared0': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]), 'have_image planet5 infrared3': array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]), 'pointing satellite4 planet5': array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), 'have_image planet3 image3': array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]), 'have_image phenomenon4 image3': array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]), 'have_image groundstation2 image0': array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]), 'pointing satellite3 planet9': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])}
18162


In [73]:
model = create_model(train_generator, 1e-5)
old_model = load_model('../model_0.h5', custom_objects = {'AttentionWeights' : goal_rec_utils.attention_layers.AttentionWeights, 'ContextVector' : goal_rec_utils.attention_layers.ContextVector, 'Custom_Hamming_Loss1' : Custom_Hamming_Loss1})
model.summary()
old_model.summary()
#model.fit(train_generator, epochs=100, validation_data=val_generator, callbacks=[SaveBestModelCallback('./', 5, 0)])

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 82)       1489366     input_11[0][0]                   
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 30, 363)      647592      embedding[0][0]                  
__________________________________________________________________________________________________
attention_weights (AttentionWei (None, 30)           393         lstm[0][0]                       
___________________________________________________________________________________________

In [70]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Custom_Hamming_Loss1, 'accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])
model.fit(train_generator, epochs=100, validation_data=val_generator, callbacks=[SaveBestModelCallback('./', 10, 0)])

Epoch 1/100
38/38 [==============================] - 8s 147ms/step - loss: 0.8004 - Custom_Hamming_Loss1: 0.1636 - accuracy: 0.1468 - precision: 0.1061 - recall: 0.0696 - val_loss: 5.4640 - val_Custom_Hamming_Loss1: 0.1138 - val_accuracy: 0.1979 - val_precision: 0.0769 - val_recall: 0.0048
Epoch 2/100
38/38 [==============================] - 5s 132ms/step - loss: 0.9375 - Custom_Hamming_Loss1: 0.1079 - accuracy: 0.1918 - precision: 0.1338 - recall: 0.0030 - val_loss: 3.6290 - val_Custom_Hamming_Loss1: 0.1115 - val_accuracy: 0.2057 - val_precision: 0.1905 - val_recall: 0.0096
Epoch 3/100
38/38 [==============================] - 5s 133ms/step - loss: 2.3048 - Custom_Hamming_Loss1: 0.1072 - accuracy: 0.3096 - precision: 0.4476 - recall: 0.0226 - val_loss: 0.8685 - val_Custom_Hamming_Loss1: 0.1117 - val_accuracy: 0.3177 - val_precision: 0.2903 - val_recall: 0.0216
Epoch 4/100
38/38 [==============================] - 5s 134ms/step - loss: 0.4502 - Custom_Hamming_Loss1: 0.0978 - accuracy: 0.

In [71]:
incremental_model_training.PlanGeneratorMultiPerc = PlanGeneratorMultiPerc
model = load_model('./model.h5', custom_objects = {'AttentionWeights' : goal_rec_utils.attention_layers.AttentionWeights, 'ContextVector' : goal_rec_utils.attention_layers.ContextVector, 'Custom_Hamming_Loss1' : Custom_Hamming_Loss1})
run_tests(model, val_plans, sub_actions_dict, sub_goals_dict, 64, 30, 0.3, 1, None)
model.save('../model_0.h5')

Accuracy: 0.19270833333333334

Hamming Loss: 0.10416666666666667

                                   precision    recall  f1-score   support

have_image phenomenon14 infrared0       0.00      0.00      0.00         2
       have_image star5 infrared3       0.00      0.00      0.00         4
     have_image planet3 infrared0       0.00      0.00      0.00         5
 have_image phenomenon4 infrared0       0.00      0.00      0.00         4
     have_image planet5 infrared3       0.00      0.00      0.00         5
      pointing satellite4 planet5       0.44      0.10      0.17        67
        have_image planet3 image3       0.58      0.25      0.35        75
    have_image phenomenon4 image3       0.55      0.29      0.38        95
 have_image groundstation2 image0       0.59      0.25      0.35        87
      pointing satellite3 planet9       0.50      0.15      0.23        67

                        micro avg       0.53      0.21      0.30       411
                        macro av

/opt/anaconda/anaconda3/envs/goal_rec/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda/anaconda3/envs/goal_rec/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [80]:
a = np.zeros([5,])
print(np.append(a, np.zeros([5,])))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
